In [182]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import json
import pandas as pd

In [ ]:
CLIENT_SECRET_FILE = 'ENTER secrets json filename'
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl', 'https://www.googleapis.com/auth/youtube', 'https://www.googleapis.com/auth/youtube.readonly','https://www.googleapis.com/auth/youtubepartner-channel-audit']
flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
credentials = flow.run_local_server(port=8080)

In [211]:
#Build API Client
youtube = build('youtube', 'v3', credentials=credentials)

In [249]:
# Define the request parameters
request = youtube.search().list(
    part='snippet',
    q='',
    type='video',
    videoCategoryId='20',#gaming video category id
    maxResults=9999999,
    regionCode='US',
    order='viewCount',
)

# Retrieve the search results
search_results = request.execute()

In [250]:
# Extract the channel IDs from the search results
channel_ids = set()
for search_result in search_results['items']:
    channel_id = search_result['snippet']['channelId']
    channel_ids.add(channel_id)
    
# Define the list to store the channel data
channel_data = []

# Define the request parameters for retrieving the channel data
for channel_id in channel_ids:
    channel_response = youtube.channels().list(
        part='snippet,statistics',
        maxResults=9999999,
        id=channel_id
    ).execute()

    # Extract the relevant data from the channel response
    channel_snippet = channel_response['items'][0]['snippet']
    channel_stats = channel_response['items'][0]['statistics']
    
    # Append the data to the channel_data list
    channel_data.append([
        channel_snippet['title'],
        channel_stats['subscriberCount'],
        channel_stats['viewCount'],
        channel_stats['videoCount'],
    ])

# Create a dataframe from the channel_data list
df = pd.DataFrame(
    channel_data,
    columns=['channel_name', 'subscribers', 'views', 'videos']
)

df[['subscribers', 'views', 'videos']] = df[['subscribers', 'views', 'videos']].apply(pd.to_numeric)

In [251]:
df.sort_values('views', ascending=False).reset_index(drop=True)

,channel_name,subscribers,views,videos
0,LankyBox,24300000,32565499970,7891
1,CookieSwirlC,19600000,22787170321,3722
2,Angry Birds,6050000,7390186132,2511
3,League of Legends,15000000,7052715079,1339
4,MrBeast Gaming,32100000,5666466616,139
5,MoreAliA,13200000,3786174604,2243
6,Nintendo of America,8980000,3196695133,5538
7,Clash Royale,9480000,1426421906,502
8,Xbox,5110000,1295331699,11723
9,Skip the Tutorial,7240000,1243926425,321
